In [1]:
# Importando e configurando módulos
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'
import jupyter2latex as j2l
from math import pi

# Projeto do modelo reduzido

O projeto do modelo reduzido é baseado na introdução de escalas que relacionem grandezas do modelo reduzido e do modelo real, onde o número de escalas deve ser menor ou igual ao número de grandezas fundamentais envolvidas no problema. Para o problema em questão as grandezas fundamentais são três: comprimento ($L$), massa ($M$) e tempo ($T$), que formam a base da matriz dimensional. Para construção do modelo reduzido as grandezas escolhidas para serem escaladas são relativas ao comprimento ($L$), à rigidez à flexão ($EI$) e à aceleração ($a$).

Os fatores de escala adotados para $L$ e $EI$ são baseados no material empregado para construção das colunas do modelo, essas serão formadas por réguas de alumínio, com cerca de $50 cm$ de comprimento, à serem adquiridas e caracterizadas. A distância entre os pavimentos será representada por trechos de $10 cm$, logo, a escala de comprimento $\lambda_L$ será:

In [2]:
L_pav = 10
scale_L = L_pav/300

j2l.print(f"$$ \lambda_L = 1:{1/scale_L:.2f} $$")

$$ \lambda_L = 1:30.00 $$

A escala da rigidez à flexão $EI$ relaciona a rigidez de 2 réguas às colunas do edifício. Adotando concreto da classe C25 tem-se $E_C=28GPa$, enquanto para a régua em alumínio tem-se $E_{r}=70GPa$. Em relação às geometrias, são estimadas para às reguas larguras de $2.5cm$ e espessuras de $0.1cm$, enquanto os pilares tem seção quadrada de $25 \times 25cm$; seus momentos de inércia $I$ são dados por \ref{eq:mI}.

\begin{equation}
\label{eq:mI}
I = \frac{bh^3}{12}
\end{equation}

Dessa forma:

In [3]:
# Propriedades dos materiais
E_C = 28E9  # Concreto
E_r = 70E9  # Régua

# Geometria em concreto dos pilares
b_C = 0.25
h_C = 0.25
I_C = b_C*h_C**3/12
EI_C = E_C * I_C
j2l.print(f'''- Momento de inércia de um pilar em concreto: ${I_C=:.3E} \: m^4$. 

- Rigidez à flexão do pilar em concreto: ${EI_C=:.3E} Nm^2$''')

# Geometria da régua
b_r = 0.025
h_r = 0.001
I_r = b_r*h_r**3/12
EI_r = E_r * I_r 
j2l.print(f'''- Momento de inércia de uma regua em alumínio: ${I_r=:.3E} \: m^4$. 

- Rigidez à flexão da régua em alumínio: ${EI_r=:.3E} Nm^2$''')


- Momento de inércia de um pilar em concreto: $I_C=3.255E-04 \: m^4$. 

- Rigidez à flexão do pilar em concreto: $EI_C=9.115E+06 Nm^2$

- Momento de inércia de uma regua em alumínio: $I_r=2.083E-12 \: m^4$. 

- Rigidez à flexão da régua em alumínio: $EI_r=1.458E-01 Nm^2$

Conforme descrito anteriormente o edifício é composto por 9 pilares, distribuídos em 3 linhas, enquanto o modelo em escala é composto por duas réguas, dessa forma, a escala da rigidez à flexão $\lambda_{EI}$ será:

In [4]:
scale_EI = (2*EI_r) / (9*EI_C)
j2l.print("$$ \lambda_{EI} =" + f"1:{1/scale_EI:.3E}$$")

$$ \lambda_{EI} =1:2.812E+08$$

Em relação à escala de acelerações ($a$) optou-se inicialmente por manter a proporção $\lambda_a=1:1$. Todavia, visto que o problema de vibração livre é independente da aceleração da gravidade essa poderá ser alterada de forma a facilitar a construção do modelo.

In [5]:
scale_a = 1/1

Em posse das escalas impostas, para construção do modelo reduzido e realização da análise devem ser conhecidas as escalas derivadas necessárias, que são obtidas através de uma análise dimensional, onde a base da nova matriz dimensional será formada pelas grandezas $L$, $EI$ e $a$. Fazendo uso da metodologia implementada por Rocha (2020), a base da matriz dimensional é alterada para a base $L$, $EI$ e $a$:

In [6]:
# Importando DimData 
DimData = pd.read_excel('../Resources/DimData.xlsx', index_col=0, sheet_name='DimData')

# Grandezas fundamentais originais 
LMT = ['L', 'M', 'T']

# Novas base de grandezas
ABC = ['L', 'EI', 'a'] 

# Importa matriz dimensional de ABC na base LMT
base = DimData.loc[ABC, LMT]
j2l.df2table(base, '{} na base {}'.format(','.join(ABC), ','.join(LMT)))

# Inverte base de unidades de LMT para ABC
base_i = pd.DataFrame(np.linalg.inv(base), index=LMT, columns=ABC)
j2l.df2table(base_i, '{} na base {}'.format(','.join(LMT), ','.join(ABC)))

,L,M,T
L,1,0,0
EI,3,1,-2
a,1,0,-2


,L,EI,a
L,1.0,0.0,0.0
M,-2.0,1.0,-1.0
T,0.5,-0.0,-0.5


Para análise do modelo reduzido é necessário o conhecimento da escala de massa ($\lambda_m$), necessária para determinação das massas dos pavimentos, e a escala de frequências ($\lambda_f$), que relaciona as frequências de vibração do modelo reduzido e da estrutura real. Para a determinação da tais fatores essas grandezas devem estar presentes na matriz dimensional de base $L$, $EI$ e $a$.

In [7]:
par = ABC + ['m', 'f']
npar = len(par)

DMat_LMT = DimData.loc[par, LMT]
DMat_ABC = np.matmul(DMat_LMT, base_i)
DMat_ABC.rename(columns=dict(zip(LMT, ABC)), inplace=True) # Renomeia colunas para nova base
j2l.df2table(DMat_ABC, 'Matriz $D$  na base {}'.format(','.join(ABC)))

,L,EI,a
L,1.0,0.0,0.0
EI,0.0,1.0,0.0
a,0.0,0.0,1.0
m,-2.0,1.0,-1.0
f,-0.5,0.0,0.5


Por fim, aplicando sobre a matriz $\bf{D}$ as escalas impostas tem-se a tabela de escalas:

In [8]:
escalas = np.array([scale_L, scale_EI, scale_a])
escalas = np.tile(escalas, (npar, 1))
escalas = np.prod(escalas**DMat_ABC, axis=1)
escalas = pd.DataFrame({'λ': escalas, '1/λ': 1/escalas}, index=par)
j2l.df2table(escalas, 'Fatores de escala')

j2l.print("""Escalas derivadas:

- Escala de massa: $\lambda_m = 1:{:.2E}$

- Escala de frequências: $\lambda_f = {:.3f}:1$""".format(escalas.loc['m', '1/λ'], escalas.loc['f', 'λ']))

,λ,1/λ
L,3.333333e-02,3.000000e+01
EI,3.555556e-09,2.812500e+08
a,1.000000e+00,1.000000e+00
m,3.200000e-06,3.125000e+05
f,5.477226e+00,1.825742e-01


Escalas derivadas:

- Escala de massa: $\lambda_m = 1:3.12E+05$

- Escala de frequências: $\lambda_f = 5.477:1$

Dessa forma a massa dos pavimentos do modelo em escala devem ter $1/312500$ da massa de cada pavimento da estrutura real. Sendo que as vigas tem seção $25\times50 cm$, a laje tem $h=10cm$ e considerando uma carga permanente (alvenarias, revestimentos e etc.) de $300 kg/m^2$ sobre todos os pavimentos, a massa dos pavimentos são:

In [9]:
M_vigas   = 0.25*0.50 * 10 * 6 * 2500   # 6 vigas de 10 metros por pavimento
M_laje    = 0.10 * 10*10 * 2500         # Laje 10x10 e h=12cm
M_cargas  = 300 * 10*10                 # carga de 300 kg/m²
M_pilares = 0.25**2 * 3 * 2500 * 9      # 9 pilares de 3m por pavimento (1.5m apenas no pav superior)

M_pavtipo = M_vigas + M_laje + M_cargas + M_pilares
M_pavcob  = M_vigas + M_laje + M_cargas + M_pilares/2

j2l.print("""Massas da estrutura:

- Massa dos pavimentos tipo: {:.1f}

- Massa do pavimento de cobertura {:.1f}""".format(M_pavtipo, M_pavcob))

Massas da estrutura:

- Massa dos pavimentos tipo: 77968.8

- Massa do pavimento de cobertura 75859.4

Logo as massas dos pavimentos do modelo em escala deverão ser:

In [10]:
j2l.print("""
Massas do modelo em escala:

- Massa dos pavimentos tipo: {:.3f}

- Massa do pavimento de cobertura {:.3f}""".format(M_pavtipo*escalas.loc['m', 'λ'], M_pavcob*escalas.loc['m', 'λ']))

Massas do modelo em escala:

- Massa dos pavimentos tipo: 0.250

- Massa do pavimento de cobertura 0.243